# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
filename = "../WeatherPy/weather_data.csv"
weather_data = pd.read_csv(filename)

weather_data.head()


,city,latitude,longitude,temp_max_F,humid,cloud,wind_mph
0,albany,42.6001,-73.9662,68.72,96,15,4.36
1,ushuaia,-54.8000,-68.3000,37.06,93,75,0.00
2,saint-philippe,-21.3585,55.7679,72.95,73,75,15.77
3,chabahar,25.2919,60.6430,87.98,80,52,6.98
4,rikitea,-23.1203,-134.9692,71.65,65,99,24.83


In [25]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        551 non-null    object 
 1   latitude    551 non-null    float64
 2   longitude   551 non-null    float64
 3   temp_max_F  551 non-null    float64
 4   humid       551 non-null    int64  
 5   cloud       551 non-null    int64  
 6   wind_mph    551 non-null    float64
dtypes: float64(4), int64(2), object(1)
memory usage: 30.3+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [83]:
# Store latitude and longitude in locations
locations = weather_data[["latitude", "longitude"]]

# use humidity as the weight
map_weight = weather_data["humid"]

# set figure layout
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px'
}

# Plot Heatmap
fig = gmaps.figure(map_type = 'HYBRID', layout=figure_layout)



# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                weights=map_weight,
                                dissipating=False, 
                                max_intensity=200,
                                point_radius=8, 
                                opacity = 0.5,
                                gradient = [
                                            'white',
                                            'aqua',
                                            'purple'
                                            ])

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', padding='1px', width='400px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [93]:
ideal_weather_places = weather_data.loc[
    (weather_data['temp_max_F']> 81) &
    (weather_data['temp_max_F']< 85) &
    (weather_data['cloud']<40) & 
    (weather_data['wind_mph']<20)]


print(len(ideal_weather_places))

8


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
